In [1]:
import pandas as pd
import os
import json
import csv
import re, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

from lifelines import KaplanMeierFitter

from global_functions import *

### Information

Input files:
data_processing/bugs_info.csv
data_processing/bug_history.csv
raw_data/all_eclipse_bugs_full.csv
raw_data/bug_history.csv


Output files:
data/bugs_info.csv
data/bugs_full.csv (full information of the final dataset)
data/bug_history.csv (injected history of the final dataset)
data/bug_history_orig.csv (original history without injection of the final dataset)
data/bugs_versions.csv (the different versions a bug was assigned to)

In [2]:
bugs_processed_info_file = 'data_processing'+os.sep+'bugs_info.zip'
bugs_processed_history_file = '.'+os.sep+'data_processing'+os.sep+'bug_history.zip'
bugs_original_history_file = '.'+os.sep+'raw_data'+os.sep+'bug_history.zip'
bugs_original_info_file = '.'+os.sep+'raw_data'+os.sep+'all_eclipse_bugs_full.zip'

bugs_final_info_file = 'data'+os.sep+'bugs_info.zip'
bugs_final_full_file = 'data'+os.sep+'bugs_full.zip'
bugs_history_final_file = 'data'+os.sep+'bug_history.zip'
bugs_history_final_file_non_injected = 'data'+os.sep+'bug_history_orig.zip'
bug_versions_file = 'data'+os.sep+'bugs_versions.csv'

In [3]:
relase_creation_ts_all = get_release_dates()

In [4]:
#get data in the preprocessing phase
df_bugs = pd.read_csv(bugs_processed_info_file,index_col=False,compression='zip'
                     dtype={'release':str,'version':str})
df_bugs = trans_to_datetime(df_bugs,['creation_time',
                             'first_resolved_date','last_resolved_date',
                             'first_fixed_date','last_fixed_date',
                             'first_assignment_date','last_assignment_date'])
df_bugs.head()

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,NaT,NaT,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30
2,475370,4.5,Platform,2015-08-19 12:09:06,4.5,DUPLICATE,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT
3,475379,4.5,Platform,2015-08-19 13:39:12,4.5,NaN,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT
4,475407,4.5,Platform,2015-08-19 17:06:10,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25


In [5]:
df_bugs['release'] = df_bugs.apply(lambda x: assign_to_closest_minor(str(x['version'])),axis=1)

In [6]:
df_bugs.version.unique()

array(['4.5', '4.6', '4.4.2', '4.4.1', '4.5.1', '3.7.2', '3.8.2', '4.4',
       '3.7.1', '4.3', '4.2', '4.2.1', '3.6.2', '4.3.2', '3.6', '4.8',
       '3.1', '4.2.2', '4.7', '3.8', '3.7', '4.3.1', '4.7.1', '4.10',
       '3.4.1', '3.4', '3.5', '3.3.2', '4.0', '3.3', '3.2.2', '3.2',
       '3.3.1', '3.0.2', '3.4.2', '3.2.1', '3.5.1', '3.1.2', '3.1.1',
       '3.0', '3.0.1', '4.1', '3.5.2', '3.6.1', '4.7.1a', '4.5.2', '4.9',
       '3.8.1', '4.7.2', '4.7.3', '3.8.0 Juno', '4.5.0 Mars',
       '3.10.0 Luna', '4.6.0 Neon', '3.10.1 Luna', '4.7.0 Oxygen',
       '3.9.0 Kepler', '3.8.2 Juno', '4.8.0 Photon', '3.8.1 Juno'],
      dtype=object)

In [7]:
df_bugs.release.unique()

array(['4.5', '4.6', '4.4', '3.7', '4.2', '4.3', '3.6', '4.8', '3.1',
       '4.7', '4.10', '3.4', '3.5', '3.3', '4.0', '3.2', '3.0', '4.1',
       '4.9', '3.10', '3.9'], dtype=object)

In [8]:
bef = len(df_bugs.id.unique())
print('Dataset size before:'+str(bef))
print('Removed releases because they are out of scope:'+str(df_bugs[~df_bugs['release'].isin(relase_creation_ts_all)].release.unique().tolist()))
df_bugs = df_bugs[df_bugs['release'].isin(relase_creation_ts_all)]
after = len(df_bugs.id.unique())
print('Removed bugs because they were not in the target releases:'+str(bef-after))
print('Dataset size:'+str(len(df_bugs.id.unique())))

Dataset size before:141010
Removed releases because they are out of scope:['4.0', '4.1', '3.10', '3.9']
Removed bugs because they were not in the target releases:2565
Dataset size:138445


In [9]:
#get the difference between the first and last date of assignment
df_bugs['time_assign_diff']=df_bugs.apply(
    lambda x: (
        x['last_assignment_date']-x['first_assignment_date']).days 
    if x['last_assignment_date'] is not None and x['first_assignment_date'] is not None 
    else None 
    ,axis=1
)

#get the difference between the first and last date of resolution
df_bugs['time_resolve_diff']=df_bugs.apply(
    lambda x: (
        x['last_resolved_date']-x['first_resolved_date']).days 
    if x['last_resolved_date'] is not None and x['first_resolved_date'] is not None 
    else None 
    ,axis=1
)

#get the difference between the first and last date of fixes
df_bugs['time_fix_diff']=df_bugs.apply(
    lambda x: (
        x['last_fixed_date']-x['first_fixed_date']).days 
    if x['last_fixed_date'] is not None and x['first_fixed_date'] is not None 
    else None 
    ,axis=1
)

df_bugs.head()

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date,time_assign_diff,time_resolve_diff,time_fix_diff
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,NaN,0.0,0.0
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,NaT,NaT,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,NaN,0.0,0.0
2,475370,4.5,Platform,2015-08-19 12:09:06,4.5,DUPLICATE,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
3,475379,4.5,Platform,2015-08-19 13:39:12,4.5,NaN,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
4,475407,4.5,Platform,2015-08-19 17:06:10,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25,NaN,0.0,0.0


In [10]:
assigned_df = df_bugs[~df_bugs['time_assign_diff'].isna()]
print('Bugs assigned only once:',len(assigned_df[assigned_df['first_assignment_date']==assigned_df['last_assignment_date']]))
print('Bugs assigned only once or assigned more than once but on the same day:',len(assigned_df[assigned_df['time_assign_diff']==0]))
print('Bugs assigned more than one time in more than one days:',len(assigned_df[assigned_df['time_assign_diff']>0]))

Bugs assigned only once: 47999
Bugs assigned only once or assigned more than once but on the same day: 48604
Bugs assigned more than one time in more than one days: 3776


In [11]:
resolved_df = df_bugs[~df_bugs['time_resolve_diff'].isna()]
print('Bugs resolved only once:',len(resolved_df[resolved_df['first_assignment_date']==resolved_df['last_assignment_date']]))
print('Bugs resolved only once or resolved more than once but on the same day:',len(resolved_df[resolved_df['time_assign_diff']==0]))
print('Bugs resolved more than one time in more than one days:',len(resolved_df[resolved_df['time_assign_diff']>0]))

Bugs resolved only once: 41466
Bugs resolved only once or resolved more than once but on the same day: 42045
Bugs resolved more than one time in more than one days: 3106


In [12]:
fixed_df = df_bugs[~df_bugs['time_fix_diff'].isna()]
print('Bugs fixed only once:',len(fixed_df[fixed_df['first_assignment_date']==fixed_df['last_assignment_date']]))
print('Bugs fixed only once or fixed more than once but on the same day:',len(fixed_df[fixed_df['time_assign_diff']==0]))
print('Bugs fixed more than one time in more than one days:',len(fixed_df[fixed_df['time_assign_diff']>0]))

Bugs fixed only once: 31170
Bugs fixed only once or fixed more than once but on the same day: 31678
Bugs fixed more than one time in more than one days: 2443


In [13]:
bug_history = pd.read_csv(bugs_processed_history_file,index_col=False,compression='zip')
bug_history = bug_history[bug_history['id'].isin(df_bugs.id.unique().tolist())]
bug_history.to_csv(bugs_history_final_file,index=False,compression='zip')

In [14]:
#Transfer the original files to the data folder with only the final dataset bugs
df_bugs_orig = pd.read_csv(bugs_original_info_file,index_col=False,compression='zip')
df_bugs_orig = df_bugs_orig[df_bugs_orig['id'].isin(df_bugs.id.unique().tolist())]
df_bugs_orig.to_csv(bugs_final_full_file,index=False,compression='zip')

bugs_history_orig  = pd.read_csv(bugs_original_history_file,index_col=False,compression='zip')
bugs_history_orig = bugs_history_orig[bugs_history_orig['id'].isin(df_bugs.id.unique().tolist())]
bugs_history_orig.to_csv(bugs_history_final_file_non_injected,index=False,compression='zip')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# "version_origin" to store the first release that the bug appeared

In [13]:
#create a new dataframe about a but present in different releases
bug_versions = pd.DataFrame()

In [14]:
bug_versions = df_bugs[['id','version']]
bug_versions['source'] = 'version field'
bug_versions.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,version,source
0,475361,4.5,version field
1,475365,4.6,version field
2,475370,4.5,version field
3,475379,4.5,version field
4,475407,4.5,version field


In [27]:
df_all = pd.read_csv(bugs_processed_history_file,index_col=False,compression='zip')
df_all[df_all['id'].isin(df_bugs.id.unique())]
df_all=trans_to_datetime(df_all,['when'])
df_all

,added,id,removed,what,when,who
0,platform-help-inbox@eclipse.org,100001,pde-ui-inbox@eclipse.org,assigned_to,2005-06-14 16:16:00,wassim.melhem@gmail.com
1,Help,100001,UI,component,2005-06-14 16:16:00,wassim.melhem@gmail.com
2,Platform,100001,PDE,product,2005-06-14 16:16:00,wassim.melhem@gmail.com
3,konradk@ca.ibm.com,100001,NaN,cc,2005-06-14 17:17:31,konradk@ca.ibm.com
4,dejan@ca.ibm.com,100001,platform-help-inbox@eclipse.org,assigned_to,2005-06-14 17:17:31,konradk@ca.ibm.com
5,3.1 RC3,100001,---,target_milestone,2005-06-14 20:26:08,konradk@ca.ibm.com
6,P2,100001,P3,priority,2005-06-14 20:44:53,dejan@ca.ibm.com
7,RESOLVED,100001,NEW,status,2005-06-14 21:26:05,dejan@ca.ibm.com
8,FIXED,100001,NaN,resolution,2005-06-14 21:26:05,dejan@ca.ibm.com
9,david_audel@fr.ibm.com,100002,jdt-core-inbox@eclipse.org,assigned_to,2006-10-06 19:42:16,Olivier_Thomann@ca.ibm.com


In [28]:
bug_versions

,id,source,version,when
139852,2382,bug history,2.0,2009-11-02 17:04:45
132067,2382,version field,3.6,NaT
140899,2802,bug history,2.0,2006-12-07 14:30:43
18483,2802,version field,3.3,NaT
142200,3091,bug history,2.0,2004-05-17 16:22:02
132068,3091,version field,3.0,NaT
142287,3109,bug history,2.0,2012-12-31 14:55:50
132069,3109,version field,3.8.0 Juno,NaT
147230,4745,bug history,2.0,2017-06-28 14:29:39
33037,4745,version field,4.8,NaT


In [29]:
version_buglist=df_all[df_all['what']=='version'].id.unique()
version_bugs=df_all[df_all['id'].isin(version_buglist)]
only_version_bugs=version_bugs[version_bugs['what']=='version']

version_bugs_grouped=only_version_bugs[['id','removed','when']].rename(index=str,columns={'removed':'version'})

version_bugs_grouped['source'] = 'bug history'

bug_versions = bug_versions.append(version_bugs_grouped,ignore_index=True)

version_bugs_grouped=only_version_bugs[['id','added','when']].rename(index=str,columns={'added':'version'})
version_bugs_grouped['source'] = 'bug history'

bug_versions = bug_versions.append(version_bugs_grouped,ignore_index=True)

bug_versions = bug_versions.drop_duplicates()
bug_versions.head()


,id,source,version,when
0,2382,bug history,2.0,2009-11-02 17:04:45
1,2382,version field,3.6,NaT
2,2802,bug history,2.0,2006-12-07 14:30:43
3,2802,version field,3.3,NaT
4,3091,bug history,2.0,2004-05-17 16:22:02


In [30]:
#if a version exists both because of the version field and the bug history, only keep the version field entry
bug_versions = bug_versions.sort_values(by=['id','version','source'])
bug_versions = bug_versions.drop_duplicates(['id','version'],keep='last')
bug_versions

,id,source,version,when
0,2382,bug history,2.0,2009-11-02 17:04:45
1,2382,version field,3.6,NaT
2,2802,bug history,2.0,2006-12-07 14:30:43
3,2802,version field,3.3,NaT
4,3091,bug history,2.0,2004-05-17 16:22:02
5,3091,version field,3.0,NaT
6,3109,bug history,2.0,2012-12-31 14:55:50
7,3109,version field,3.8.0 Juno,NaT
8,4745,bug history,2.0,2017-06-28 14:29:39
9,4745,version field,4.8,NaT


In [31]:
bug_versions.version.unique()

array(['2.0', '3.6', '3.3', '3.0', '3.8.0 Juno', '4.8', '3.2', '3.1',
       'unspecified', '2.1.2', '3.5', '2.1', '2.0.2', '1.0', '1.1',
       '2.1.1', '1.2', '3.1.2', '3.8', '3.0.1', '3.4', '2.1.3', '1.1.3',
       '2.0.1', '3.7', '1.0.0', '3.0.2', '4.0', '3.1.1', '4.7', '0.7',
       '3.2.1', '2.2', '4.4', '4.5.1', '3.2.2', '4.1', '4.4.2', '1.3.0',
       '1.5', '1.0.1', '3.4.1', '1.0.2', '4.2', '2.0.0', '3.3.1', '0.7.1',
       '0.9', '4.2.2', '2.1.0', '3.6.1', '4.3', '0.2', '0.6', '0.8',
       '4.5', '1.2.1', '1.5.2', '3.3.2', '1.5.3', '0.1.3', '1.5.4',
       '4.0.1', '4.6', '3.4.2', '4.0.2', '2.4.0', '1.5.5', 'dev', '2.3.0',
       '2.3', '5.0', '4.0.3', '3.5.1', '1.3.1', '3.0.3', '2.5.0', '3.6.2',
       '2.3.1', '1.0.3', '3.5.2', '5.0.1', '2008-Ganymede', '0.1', '6.0',
       'DD 1.1', '5.0.2', '3.0.0', '1.6.4', '0.9.2', '1.7', '1.1.0',
       '3.7.2', '1.2.0', 'Europa', 'Galileo', '1.7.1', '1.8', '4.2.1',
       '3.8.1', '0.8.0', '6.0.2', '2.6.0', '0 DD 1.1', '3.8.2', '7.0'

In [33]:
#filter 1: fix names that we know the number for, e.g., Galileo
bug_versions.loc[bug_versions['version']=='Europa','version'] = '3.3'
bug_versions.loc[bug_versions['version']=='2008-Ganymede','version'] = '3.4'
bug_versions.loc[bug_versions['version']=='Galileo','version'] = '3.5'


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [34]:
#filter 2: 
bug_versions = bug_versions[bug_versions['version'].str.find('.')>=0]
bug_versions.head()


,id,source,version,when
0,2382,bug history,2.0,2009-11-02 17:04:45
1,2382,version field,3.6,NaT
2,2802,bug history,2.0,2006-12-07 14:30:43
3,2802,version field,3.3,NaT
4,3091,bug history,2.0,2004-05-17 16:22:02


In [35]:
bug_versions.version.unique()

array(['2.0', '3.6', '3.3', '3.0', '3.8.0 Juno', '4.8', '3.2', '3.1',
       '2.1.2', '3.5', '2.1', '2.0.2', '1.0', '1.1', '2.1.1', '1.2',
       '3.1.2', '3.8', '3.0.1', '3.4', '2.1.3', '1.1.3', '2.0.1', '3.7',
       '1.0.0', '3.0.2', '4.0', '3.1.1', '4.7', '0.7', '3.2.1', '2.2',
       '4.4', '4.5.1', '3.2.2', '4.1', '4.4.2', '1.3.0', '1.5', '1.0.1',
       '3.4.1', '1.0.2', '4.2', '2.0.0', '3.3.1', '0.7.1', '0.9', '4.2.2',
       '2.1.0', '3.6.1', '4.3', '0.2', '0.6', '0.8', '4.5', '1.2.1',
       '1.5.2', '3.3.2', '1.5.3', '0.1.3', '1.5.4', '4.0.1', '4.6',
       '3.4.2', '4.0.2', '2.4.0', '1.5.5', '2.3.0', '2.3', '5.0', '4.0.3',
       '3.5.1', '1.3.1', '3.0.3', '2.5.0', '3.6.2', '2.3.1', '1.0.3',
       '3.5.2', '5.0.1', '0.1', '6.0', 'DD 1.1', '5.0.2', '3.0.0',
       '1.6.4', '0.9.2', '1.7', '1.1.0', '3.7.2', '1.2.0', '1.7.1', '1.8',
       '4.2.1', '3.8.1', '0.8.0', '6.0.2', '2.6.0', '0 DD 1.1', '3.8.2',
       '7.0', '3.7.1', '4.3.1', '4.3.2', '3.0.5', '2.2.1', '8.0', '0.5',

In [36]:
#fix abnormal values because we might miss closest minor releases
for versioninit in bug_versions.version.unique().tolist():
    version = versioninit
    if ' ' in version:
        if '.' not in version[:version.find(' ')]:
            continue
        else:
            version = version[:version.find(' ')]
            bug_versions.loc[bug_versions['version']==versioninit,'version'] = version

bug_versions.version.unique()

array(['2.0', '3.6', '3.3', '3.0', '3.8.0', '4.8', '3.2', '3.1', '2.1.2',
       '3.5', '2.1', '2.0.2', '1.0', '1.1', '2.1.1', '1.2', '3.1.2',
       '3.8', '3.0.1', '3.4', '2.1.3', '1.1.3', '2.0.1', '3.7', '1.0.0',
       '3.0.2', '4.0', '3.1.1', '4.7', '0.7', '3.2.1', '2.2', '4.4',
       '4.5.1', '3.2.2', '4.1', '4.4.2', '1.3.0', '1.5', '1.0.1', '3.4.1',
       '1.0.2', '4.2', '2.0.0', '3.3.1', '0.7.1', '0.9', '4.2.2', '2.1.0',
       '3.6.1', '4.3', '0.2', '0.6', '0.8', '4.5', '1.2.1', '1.5.2',
       '3.3.2', '1.5.3', '0.1.3', '1.5.4', '4.0.1', '4.6', '3.4.2',
       '4.0.2', '2.4.0', '1.5.5', '2.3.0', '2.3', '5.0', '4.0.3', '3.5.1',
       '1.3.1', '3.0.3', '2.5.0', '3.6.2', '2.3.1', '1.0.3', '3.5.2',
       '5.0.1', '0.1', '6.0', 'DD 1.1', '5.0.2', '3.0.0', '1.6.4',
       '0.9.2', '1.7', '1.1.0', '3.7.2', '1.2.0', '1.7.1', '1.8', '4.2.1',
       '3.8.1', '0.8.0', '6.0.2', '2.6.0', '0 DD 1.1', '3.8.2', '7.0',
       '3.7.1', '4.3.1', '4.3.2', '3.0.5', '2.2.1', '8.0', '0.5', '7.0

In [37]:
bug_versions['release'] = bug_versions.apply(lambda x: assign_to_closest_minor(str(x['version'])),axis=1)
bug_versions['release'] = bug_versions['release'].astype(str)
bug_versions

,id,source,version,when,release
0,2382,bug history,2.0,2009-11-02 17:04:45,2.0
1,2382,version field,3.6,NaT,3.6
2,2802,bug history,2.0,2006-12-07 14:30:43,2.0
3,2802,version field,3.3,NaT,3.3
4,3091,bug history,2.0,2004-05-17 16:22:02,2.0
5,3091,version field,3.0,NaT,3.0
6,3109,bug history,2.0,2012-12-31 14:55:50,2.0
7,3109,version field,3.8.0,NaT,4.2
8,4745,bug history,2.0,2017-06-28 14:29:39,2.0
9,4745,version field,4.8,NaT,4.8


In [28]:
bug_versions.head()

,id,version,source,release
139852,2382,2.0,bug history,2.0
132067,2382,3.6,version field,3.6
140899,2802,2.0,bug history,2.0
18483,2802,3.3,version field,3.3
142200,3091,2.0,bug history,2.0


In [38]:
print('Current releases:'+str(sorted(bug_versions.release.unique().tolist())))

Current releases:['0 DD 0.9', '0 DD 1.1', '0.1', '0.11', '0.12', '0.13', '0.14', '0.15', '0.17', '0.2', '0.21', '0.22', '0.24', '0.3', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0', '1.1', '1.10', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '2.0', '2.1', '2.11', '2.12', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '2.9', '3.0', '3.1', '3.10', '3.11', '3.13', '3.14', '3.16', '3.18', '3.19', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.9', '4.0', '4.1', '4.10', '4.11', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5.0', '5.1', '5.2', '5.9', '6.0', '7.0', '8.0', '8.1', '8.2', '8.3', '8.4', '8.5', '8.6', '8.7', '8.8', '9.0', '9.2', '9.3', '9.4', '9.5', 'DD 1.1']


In [39]:
#filter 3: exclude the releases not in our study
bug_versions = bug_versions[bug_versions['release'].isin(relase_creation_ts_all)]
print('After filtering releases:'+str(sorted(bug_versions.release.unique().tolist())))

After filtering releases:['3.0', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '4.10', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9']


In [46]:
len(bug_versions[bug_versions['source']!='version field'].id.unique())

4160

In [ ]:
counter1=0
counter2=0
for bid in bug_versions[bug_versions['source']!='version field'].id.unique():
    try:
        version_field_release = bug_versions[(bug_versions['id']==bid )& (bug_versions['source']=='version field')].release.values.tolist()[0]
    except:
        continue
    history_values = bug_versions[(bug_versions['id']==bid )& (bug_versions['source']=='bug history')].release.values.tolist()
    
    major_change = [x for x in history_values if x!=version_field_release]
    if len(major_change)>0:
        counter1+=1
        rf = version_field_release[:1]
        ff = [x for x in major_change if x[:1]==rf]
        if len(ff)>0:
            counter2+=1
print('counter1='+str(counter1))
print('counter1='+str(counter2))

In [73]:
df_bugs[df_bugs['id'].isin(sbugs)].sort_values(by='id')

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date,time_assign_diff,time_resolve_diff,time_fix_diff
110228,39064,3.2,JDT,2003-06-18 12:31:08,3.2,WORKSFORME,0,1,0,NaT,NaT,2006-05-24 20:37:12,2006-05-24 20:37:12,NaT,NaT,NaN,0.0,NaN
110272,39447,3.2,JDT,2003-06-27 23:00:00,3.2,INVALID,0,1,0,NaT,NaT,2006-05-24 20:43:42,2006-05-24 20:44:21,NaT,NaT,NaN,0.0,NaN
133913,39593,3.3,Equinox,2003-07-03 11:45:13,3.3,WONTFIX,0,1,0,NaT,NaT,2010-04-12 14:35:01,2010-04-12 14:35:01,NaT,NaT,NaN,0.0,NaN
87328,39955,3.2,JDT,2003-07-11 19:16:14,3.2,WONTFIX,0,1,0,NaT,NaT,2006-05-24 20:48:08,2006-05-24 20:48:08,NaT,NaT,NaN,0.0,NaN
87652,41618,3.2,JDT,2003-08-15 15:44:16,3.2,WONTFIX,0,1,0,NaT,NaT,2006-05-24 19:54:21,2006-05-30 08:34:55,NaT,NaT,NaN,5.0,NaN
87660,41652,3.2,JDT,2003-08-18 15:53:50,3.2,FIXED,0,1,1,NaT,NaT,2006-05-24 20:49:55,2006-05-24 20:49:55,2006-05-24 20:49:55,2006-05-24 20:49:55,NaN,0.0,0.0
87704,41858,3.2,JDT,2003-08-22 13:49:27,3.2,NaN,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
87707,41871,3.2,JDT,2003-08-22 17:08:49,3.2,WONTFIX,0,1,0,NaT,NaT,2006-05-24 20:52:21,2006-05-24 20:52:21,NaT,NaT,NaN,0.0,NaN
85549,43136,3.2,JDT,2003-09-16 09:25:04,3.2,WORKSFORME,0,1,0,NaT,NaT,2006-05-24 20:57:49,2006-05-24 20:57:49,NaT,NaT,NaN,0.0,NaN
10016,43386,3.1,Platform,2003-09-19 21:45:19,3.1.2,FIXED,0,1,1,NaT,NaT,2004-10-18 19:40:51,2007-04-13 16:49:18,2004-10-18 19:40:51,2007-04-13 16:49:18,NaN,906.0,906.0


In [74]:
bug_versions[(bug_versions['id'].isin(sbugs)) &
            (bug_versions['source']=='bug history') &
            (bug_versions['release']=='3.0')]

,id,source,version,when,release
264,39064,bug history,3.0,2003-06-18 12:31:20,3.0
394,39447,bug history,3.0,2006-05-24 09:48:23,3.0
429,39593,bug history,3.0,2007-06-20 20:58:56,3.0
602,39955,bug history,3.0,2006-05-24 09:48:24,3.0
1475,41618,bug history,3.0,2006-05-23 22:46:16,3.0
1493,41652,bug history,3.0,2006-05-24 09:48:25,3.0
1611,41858,bug history,3.0,2006-05-24 09:48:25,3.0
1623,41871,bug history,3.0,2006-05-24 09:48:25,3.0
2351,43136,bug history,3.0,2006-05-24 09:48:26,3.0
2488,43386,bug history,3.0,2007-04-09 18:57:48,3.0


In [70]:
sbugs = bug_versions[bug_versions['source']!='version field'].id.unique().tolist()
sbugs

[39064,
 39447,
 39593,
 39955,
 41618,
 41652,
 41858,
 41871,
 43136,
 43386,
 46339,
 47354,
 47403,
 49557,
 50215,
 50362,
 51151,
 51580,
 52802,
 56447,
 57017,
 57349,
 57570,
 57699,
 58185,
 58194,
 59342,
 61190,
 61678,
 61744,
 62760,
 63433,
 63514,
 64310,
 64797,
 64878,
 65088,
 65089,
 65796,
 65797,
 66556,
 66719,
 66758,
 67418,
 67500,
 67539,
 67543,
 67654,
 67791,
 67810,
 68330,
 68518,
 68632,
 69753,
 69892,
 70238,
 70579,
 70875,
 70891,
 71055,
 71098,
 71288,
 71304,
 71636,
 71739,
 71783,
 72053,
 72092,
 72191,
 72316,
 72507,
 72643,
 73460,
 73487,
 73784,
 73977,
 74119,
 74406,
 74594,
 74709,
 74722,
 74936,
 75128,
 75153,
 75272,
 75360,
 75409,
 75496,
 75602,
 75641,
 75666,
 75785,
 75899,
 75930,
 75943,
 75958,
 75962,
 75990,
 76036,
 76108,
 76245,
 76249,
 76256,
 76295,
 76327,
 76446,
 76497,
 76498,
 76501,
 76784,
 76814,
 76832,
 76992,
 76993,
 77026,
 77146,
 77283,
 77284,
 77355,
 77371,
 77388,
 77456,
 77496,
 77563,
 77567,


In [ ]:
bug_versions.to_csv(bug_versions_file,index=False,compression='zip')

In [32]:
def get_tap(value):
    tap = tuple(int(i) for i in value.split('.'))
    return tap

def ag_function_min(values):
    values = values.unique().tolist()
    if len(values)>1:
        values = [get_tap(x) for x in values]
        values = sorted(values)
        min_val = values[0]
        return str(min_val[0])+'.'+str(min_val[1])
    else:
        return values[0]
    
def ag_function_max(values):
    values = values.unique().tolist()
    if len(values)>1:
        values = [get_tap(x) for x in values]
        values = sorted(values,reverse=True)
        max_val = values[0]
        return str(max_val[0])+'.'+str(max_val[1])
    else:
        return values[0]

min_releases = (
    bug_versions
    .groupby(by=['id'])
    .agg({'release':ag_function_min})
    .reset_index()
    .rename(index=str,columns={'release':'min_release'})
)
max_releases = (
    bug_versions
    .groupby(by=['id'])
    .agg({'release':ag_function_max})
    .reset_index()
    .rename(index=str,columns={'release':'max_release'})
)

if 'min_release' not in df_bugs.columns:
    df_bugs = pd.merge(df_bugs,min_releases,on=['id'],how='left')
    
if 'max_release' not in df_bugs.columns:
    df_bugs = pd.merge(df_bugs,max_releases,on=['id'],how='left')

In [33]:
df_bugs.head()

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date,time_assign_diff,time_resolve_diff,time_fix_diff,min_release,max_release
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,2015-08-19 11:53:03,NaN,0.0,0.0,4.5,4.5
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,NaT,NaT,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,2015-08-26 14:29:30,NaN,0.0,0.0,4.6,4.6
2,475370,4.5,Platform,2015-08-19 12:09:06,4.5,DUPLICATE,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,4.5,4.5
3,475379,4.5,Platform,2015-08-19 13:39:12,4.5,NaN,0,0,0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,4.5,4.5
4,475407,4.5,Platform,2015-08-19 17:06:10,4.5,FIXED,0,1,1,NaT,NaT,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25,2015-08-19 17:15:25,NaN,0.0,0.0,4.5,4.5


In [34]:
#the bugs with not version coming from the bug history will have None values in the min and max release columns
for index,row in df_bugs[df_bugs['min_release'].isna()].iterrows():
    df_bugs.loc[index,'min_release'] = row['release']
    
for index,row in df_bugs[df_bugs['max_release'].isna()].iterrows():
    df_bugs.loc[index,'max_release'] = row['release']


In [35]:
print('Bugs in more than one release:'+str(len(df_bugs[df_bugs['min_release']!=df_bugs['max_release']])))
perc = len(df_bugs[df_bugs['min_release']!=df_bugs['max_release']])/len(df_bugs)
print('Percentage:'+str(perc))

Bugs in more than one release:3604
Percentage:0.026031998266459605


In [36]:
#create a temp dataframe to calculate this
dftemp = df_bugs
dftemp['min_release_before_version_field'] = dftemp.apply(
    lambda x :
    is_larger_release(x['release'],x['min_release']),
    axis=1
)
print('Bugs started in earlier releases (compared to the verion field value):'+str(len(dftemp[dftemp['min_release_before_version_field']==True])))

Bugs started in earlier releases (compared to the verion field value):1709


In [37]:
dftemp['min_release_after_version_field'] = dftemp.apply(
    lambda x :
    is_larger_release(x['min_release'],x['release']),
    axis=1
)
print('Bugs started in subsequent releases (compared to the verion field value):'+str(len(dftemp[dftemp['min_release_after_version_field']==True])))

Bugs started in subsequent releases (compared to the verion field value):0


In [38]:
#EXPORT FILES
df_bugs.to_csv(bugs_final_info_file,index=False,compression='zip')

# Take into account if the release is changed when calculating the difference between the first and last assingment date

### WILL WE DO THIS???? IF NOT, REMOVE THE SUBSEQUENT CELLS

In [ ]:
@ELENI: CONTINUE FROM HERE

In [ ]:
df_all = pd.read_csv('.'+os.sep+'data'+os.sep+'bugs_info_initial_versions.zip',index_col=False,compression='zip')
df_all = df_bugs.fillna('')
df_all.head()

In [ ]:
#@Zeinab: what does this count?
assigned_in_zero=assigned_df[assigned_df['f_l_assign_diff']!=pd.Timedelta(0)]

assigned_in_zero=assigned_in_zero[['bug_id']]
assigned_in_zero.columns

vers_bugs=pd.merge(assigned_in_zero, df_all, on=['bug_id'], how='inner')

len(vers_bugs.bug_id.unique())


vers=vers_bugs[vers_bugs['version']!=vers_bugs['intial_version']]


version_change=pd.merge(vers, df, on=['bug_id', 'release', 'Product', 'creation_ts', 'version'], how='inner')


len(version_change.bug_id.unique())

In [ ]:
version_change=version_change[version_change['What']=='version']
version_change['last_assignment_date'] = pd.to_datetime(version_change['last_assignment_date'])
version_change['resolved_date'] = pd.to_datetime(version_change['resolved_date'])


In [ ]:
version_btw_l_f=version_change[version_change['When']<version_change['last_assignment_date']]
len(version_btw_l_f.bug_id.unique())

In [ ]:
version_btw_l_f=version_btw_l_f[version_btw_l_f['When']>version_btw_l_f['resolved_date']]
len(version_btw_l_f.bug_id.unique())

In [ ]:
version_btw_l_f['added_minor']=version_btw_l_f.apply(lambda x: str(closest_minor(x['Added'])),axis=1)
version_btw_l_f['removed_minor']=version_btw_l_f.apply(lambda x: str(closest_minor(x['removed'])),axis=1)

In [ ]:
version_btw_l_f[version_btw_l_f['added_minor']!=version_btw_l_f['removed_minor']]

In [ ]:

version_btw_l_f[version_btw_l_f['bug_id']==246547]